Prior to loading this data into jupyter notebook first I created a SQLite database and inserted the 3 (airlines, airports & flights) tables into with the following steps in SQL command prompt:

To check for existing dbs: > .databases
To open or create db > .open DATABASE_NAME.db
To import csv > .mode csv
 .import CSV_NAME.csv TABLENAME
To check tables in db > .tables
To view table content > . mode columns
 .header on
SELECT * FROM TABLENAME;
To save be db > .backup DATABASE_NAME.db



In [1]:
#import dependencies
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [2]:
database_path = "flights.db"
engine = create_engine(f"sqlite:///{database_path}")
conn = engine.connect()

In [33]:
database_path = "etl_1"
engine = create_engine(f"sqlite:///{database_path}")
conn = engine.connect()

In [3]:
#Double check it worked
inspector = inspect(engine)
inspector.get_table_names()

['airlines',
 'airports',
 'avg_flight_delay_with_names',
 'cancelled_and_count',
 'flights',
 'total_cancelled',
 'total_flights']

In [29]:
# Get a list of column names and types
columns = inspector.get_columns('airlines')
for c in columns:
    print(c['name'], c["type"])
# columns

IATA_CODE TEXT
AIRLINE TEXT


In [30]:
# Get a list of column names and types
columns = inspector.get_columns('airports')
for c in columns:
    print(c['name'], c["type"])
# columns

IATA_CODE TEXT
AIRPORT TEXT
CITY TEXT
STATE TEXT
COUNTRY TEXT
LATITUDE TEXT
LONGITUDE TEXT


In [26]:
# Get a list of column names and types
columns = inspector.get_columns('flights')
for c in columns:
    print(c['name'], c["type"])
# columns

YEAR TEXT
MONTH TEXT
DAY TEXT
DAY_OF_WEEK TEXT
AIRLINE TEXT
FLIGHT_NUMBER TEXT
TAIL_NUMBER TEXT
ORIGIN_AIRPORT TEXT
DESTINATION_AIRPORT TEXT
SCHEDULED_DEPARTURE TEXT
DEPARTURE_TIME TEXT
DEPARTURE_DELAY TEXT
TAXI_OUT TEXT
WHEELS_OFF TEXT
SCHEDULED_TIME TEXT
ELAPSED_TIME TEXT
AIR_TIME TEXT
DISTANCE TEXT
WHEELS_ON TEXT
TAXI_IN TEXT
SCHEDULED_ARRIVAL TEXT
ARRIVAL_TIME TEXT
ARRIVAL_DELAY TEXT
DIVERTED TEXT
CANCELLED TEXT
CANCELLATION_REASON TEXT
AIR_SYSTEM_DELAY TEXT
SECURITY_DELAY TEXT
AIRLINE_DELAY TEXT
LATE_AIRCRAFT_DELAY TEXT
WEATHER_DELAY TEXT


In [34]:
pd.read_sql_query('select * from flights limit 10', con=engine).tail(20)

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,None,None,None,None,None,None
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,None,None,None,None,None,None
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,None,None,None,None,None,None
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,None,None,None,None,None,None
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,None,None,None,None,None,None
5,2015,1,1,4,DL,806,N3730B,SFO,MSP,25,...,610.0,8.0,0,0,None,None,None,None,None,None
6,2015,1,1,4,NK,612,N635NK,LAS,MSP,25,...,509.0,-17.0,0,0,None,None,None,None,None,None
7,2015,1,1,4,US,2013,N584UW,LAX,CLT,30,...,753.0,-10.0,0,0,None,None,None,None,None,None
8,2015,1,1,4,AA,1112,N3LAAA,SFO,DFW,30,...,532.0,-13.0,0,0,None,None,None,None,None,None
9,2015,1,1,4,DL,1173,N826DN,LAS,ATL,30,...,656.0,-15.0,0,0,None,None,None,None,None,None


In [11]:
len(flights)

NameError: name 'flights' is not defined

In [35]:
#check out the average arrival delay on airline to see what airlines have the worst delays
avg_flight_delay = pd.read_sql_query('SELECT AIRLINE as airline_code, AVG(ARRIVAL_DELAY) as average_delay FROM flights  GROUP BY AIRLINE ORDER BY AVG(ARRIVAL_DELAY) desc LIMIT 0,30000000;', con=engine)
avg_flight_delay

,airline_code,average_delay
0,NK,14.471800
1,F9,12.504706
2,B6,6.677861
3,EV,6.585379
4,MQ,6.457873
5,OO,5.845652
6,UA,5.431594
7,VX,4.737706
8,WN,4.374964
9,US,3.706209


In [6]:
#insert this table into the sqlite database to perform further queries
avg_flight_delay.to_sql(name='avg_flight_delay', con=engine, if_exists='replace', index=False)

In [7]:
#check it worked
engine.table_names()

['airlines',
 'airports',
 'avg_flight_delay',
 'avg_flight_delay_with_names',
 'cancelled_and_count',
 'flights',
 'total_cancelled',
 'total_flights']

In [72]:
#combine the avg delay to show the actual airline as opposed to the shortcode only 
avg_flight_delay_with_names = pd.read_sql_query('SELECT airlines.AIRLINE, avg_flight_delay.airline_code, avg_flight_delay.average_delay\
                                     FROM airlines \
                                     JOIN avg_flight_delay \
                                     ON airlines.IATA_code = avg_flight_delay.airline_code\
                                     ORDER BY(average_delay) desc;', con = engine)

avg_flight_delay_with_names

,AIRLINE,airline_code,average_delay
0,Spirit Air Lines,NK,14.202285
1,Frontier Airlines Inc.,F9,12.402010
2,JetBlue Airways,B6,6.552680
3,Atlantic Southeast Airlines,EV,6.387061
4,American Eagle Airlines Inc.,MQ,6.110663
5,Skywest Airlines Inc.,OO,5.731005
6,United Air Lines Inc.,UA,5.347749
7,Virgin America,VX,4.687576
8,Southwest Airlines Co.,WN,4.307522
9,US Airways Inc.,US,3.622429


In [73]:
#insert this table into the sqlite database to perform further queries
avg_flight_delay_with_names.to_sql(name='avg_flight_delay_with_names', con=engine, if_exists='replace', index=False)
#drop the previous table to keep the database clean
engine.execute("DROP TABLE avg_flight_delay")
#check it worked
engine.table_names()

['airlines',
 'airports',
 'avg_flight_delay_with_names',
 'cancelled_and_count',
 'flights',
 'total_cancelled',
 'total_flights']

In [50]:
#combine the avg delay to show the actual airline as opposed to the shortcode only 
cancelled_count = pd.read_sql_query('SELECT airlines.AIRLINE, airlines.IATA_CODE, flights.CANCELLED as total_cancelled\
                                     FROM airlines \
                                     JOIN flights \
                                     ON airlines.IATA_code = flights.AIRLINE\
                                     WHERE flights.CANCELLED = 1;', con = engine)

#insert this table into the sqlite database to perform further queries
cancelled_count.to_sql(name='cancelled_count', con=engine, if_exists='replace', index=False)
#check it worked
engine.table_names()

['airlines',
 'airports',
 'avg_flight_delay_with_names',
 'cancelled_count',
 'flights',
 'total_cancelled',
 'total_flights']

In [51]:
cancelled_count.head()

,AIRLINE,IATA_CODE,total_cancelled
0,Alaska Airlines Inc.,AS,1
1,American Airlines Inc.,AA,1
2,Skywest Airlines Inc.,OO,1
3,American Eagle Airlines Inc.,MQ,1
4,Skywest Airlines Inc.,OO,1


In [52]:
#Get an idea on which airlines cancel the most flights
total_cancelled = pd.read_sql_query('SELECT AIRLINE, IATA_CODE, SUM(total_cancelled) as cancelled_flights\
                                     FROM cancelled_count \
                                     GROUP BY AIRLINE\
                                     ORDER BY SUM(total_cancelled) desc;', con = engine)
total_cancelled

,AIRLINE,IATA_CODE,cancelled_flights
0,Southwest Airlines Co.,WN,16043
1,Atlantic Southeast Airlines,EV,15231
2,American Eagle Airlines Inc.,MQ,15025
3,American Airlines Inc.,AA,10919
4,Skywest Airlines Inc.,OO,9960
5,United Air Lines Inc.,UA,6573
6,JetBlue Airways,B6,4276
7,US Airways Inc.,US,4067
8,Delta Air Lines Inc.,DL,3824
9,Spirit Air Lines,NK,2004


In [53]:
#insert this table into the sqlite database to perform further queries
total_cancelled.to_sql(name='total_cancelled', con=engine, if_exists='replace', index=False)
#drop the previous table to keep the database clean
engine.execute("DROP TABLE cancelled_count")
#check it worked
engine.table_names()

['airlines',
 'airports',
 'avg_flight_delay_with_names',
 'flights',
 'total_cancelled',
 'total_flights']

In [54]:
#get the total count of flights to get a ratio of total cancelled
#Get an idea on which airlines cancel the most flights
total_flights = pd.read_sql_query("SELECT AIRLINE, COUNT(AIRLINE) as flight_totals\
                                     FROM flights \
                                     GROUP BY AIRLINE", con = engine)
#insert this table into the sqlite database to perform further queries
total_flights.to_sql(name='total_flights', con=engine, if_exists='replace', index=False)
#check it worked
engine.table_names()

['airlines',
 'airports',
 'avg_flight_delay_with_names',
 'flights',
 'total_cancelled',
 'total_flights']

In [56]:
total_flights

,AIRLINE,flight_totals
0,AA,725984
1,AS,172521
2,B6,267048
3,DL,875881
4,EV,571977
5,F9,90836
6,HA,76272
7,MQ,294632
8,NK,117379
9,OO,588353


In [59]:
#merge the total cancellation and total number of flights to get the cancellation ratio
#combine the avg delay to show the actual airline as opposed to the shortcode only 
cancelled_and_count = pd.read_sql_query("SELECT total_cancelled.AIRLINE, total_cancelled.IATA_CODE, total_cancelled.cancelled_flights,total_flights.flight_totals FROM total_cancelled JOIN total_flights ON total_cancelled.IATA_CODE = total_flights.AIRLINE", con = engine)


In [63]:
#clean up in pandas for ratio
cancelled_and_count['cancellation ratio'] = round(cancelled_and_count['cancelled_flights'] / cancelled_and_count['flight_totals'] *100,2)


,AIRLINE,IATA_CODE,cancelled_flights,flight_totals,cancellation ratio
0,Southwest Airlines Co.,WN,16043,1261855,1.27
1,Atlantic Southeast Airlines,EV,15231,571977,2.66
2,American Eagle Airlines Inc.,MQ,15025,294632,5.10
3,American Airlines Inc.,AA,10919,725984,1.50
4,Skywest Airlines Inc.,OO,9960,588353,1.69
5,United Air Lines Inc.,UA,6573,515723,1.27
6,JetBlue Airways,B6,4276,267048,1.60
7,US Airways Inc.,US,4067,198715,2.05
8,Delta Air Lines Inc.,DL,3824,875881,0.44
9,Spirit Air Lines,NK,2004,117379,1.71


In [65]:
cancelled_and_count = cancelled_and_count.sort_values(['cancellation ratio'])

In [67]:
#insert this table into the sqlite database to perform further queries
cancelled_and_count.to_sql(name='cancelled_and_count', con=engine, if_exists='replace', index=False)
#check it worked
engine.table_names()

['airlines',
 'airports',
 'avg_flight_delay_with_names',
 'cancelled_and_count',
 'flights',
 'total_cancelled',
 'total_flights']

In [12]:
query_string = 'SELECT a.AIRLINE as `Airline Name`, a.IATA_CODE as `Airline Code`, \
                   COUNT(f.AIRLINE) as `Total Flights`, \
                   AVG(f.DEPARTURE_DELAY) as `Avg DEPARTURE_DELAY`, SUM(f.ELAPSED_TIME) as `Sum ELAPSED_TIME`, \
                   AVG(f.ELAPSED_TIME) as `Avg ELAPSED_TIME`, AVG(f.TAXI_OUT) as `Avg TAXI_OUT`, \
                   SUM(f.AIR_TIME) as `Sum AIR_TIME`, \
                   AVG(f.AIR_TIME) as `Avg AIR_TIME`, \
                   AVG(f.TAXI_IN) as `Avg TAXI_IN`, \
                   SUM(f.DISTANCE) as `Sum DISTANCE`, \
                   AVG(f.DISTANCE) as `Avg DISTANCE`, \
                   AVG(f.ARRIVAL_DELAY) as `Avg ARRIVAL_DELAY`, SUM(f.DIVERTED) as `DIVERTED count`, \
                   SUM(f.CANCELLED) as `CANCELLED count` FROM airlines as a \
               INNER JOIN flights as f ON a.IATA_CODE = f.AIRLINE \
               GROUP BY f.AIRLINE'

In [13]:
query_string

'SELECT a.AIRLINE as `Airline Name`, a.IATA_CODE as `Airline Code`,                    COUNT(f.AIRLINE) as `Total Flights`,                    AVG(f.DEPARTURE_DELAY) as `Avg DEPARTURE_DELAY`, SUM(f.ELAPSED_TIME) as `Sum ELAPSED_TIME`,                    AVG(f.ELAPSED_TIME) as `Avg ELAPSED_TIME`, AVG(f.TAXI_OUT) as `Avg TAXI_OUT`,                    SUM(f.AIR_TIME) as `Sum AIR_TIME`,                    AVG(f.AIR_TIME) as `Avg AIR_TIME`,                    AVG(f.TAXI_IN) as `Avg TAXI_IN`,                    SUM(f.DISTANCE) as `Sum DISTANCE`,                    AVG(f.DISTANCE) as `Avg DISTANCE`,                    AVG(f.ARRIVAL_DELAY) as `Avg ARRIVAL_DELAY`, SUM(f.DIVERTED) as `DIVERTED count`,                    SUM(f.CANCELLED) as `CANCELLED count` FROM airlines as a                INNER JOIN flights as f ON a.IATA_CODE = f.AIRLINE                GROUP BY f.AIRLINE'

In [14]:
#create the megaview for future reference
df_airline_delay = pd.read_sql_query(query_string, con=engine)
df_airline_delay

,Airline Name,Airline Code,Total Flights,Avg DEPARTURE_DELAY,Sum ELAPSED_TIME,Avg ELAPSED_TIME,Avg TAXI_OUT,Sum AIR_TIME,Avg AIR_TIME,Avg TAXI_IN,Sum DISTANCE,Avg DISTANCE,Avg ARRIVAL_DELAY,DIVERTED count,CANCELLED count
0,American Airlines Inc.,AA,725984,8.773520,118656112.0,163.441773,17.503434,99672739.0,137.293300,8.740104,755995614,1041.339222,3.389336,2130,10919
1,Alaska Airlines Inc.,AS,172521,1.779476,30745834.0,178.215023,15.037943,27060752.0,156.854829,6.370906,206579765,1197.418082,-0.970438,413,669
2,JetBlue Airways,B6,267048,11.333045,43980830.0,164.692602,17.586947,37696967.0,141.161765,6.021408,283651757,1062.175178,6.552680,730,4276
3,Delta Air Lines Inc.,DL,875881,7.338090,122249239.0,139.572886,17.531950,100660665.0,114.925047,7.176670,747671138,853.621825,0.185558,1782,3824
4,Atlantic Southeast Airlines,EV,571977,8.492191,53879870.0,94.199365,16.294799,40365887.0,70.572570,7.427902,264397363,462.251739,6.387061,1994,15231
5,Frontier Airlines Inc.,F9,90836,13.270609,13969056.0,153.783258,15.572735,11733373.0,129.170956,9.086320,87857929,967.214860,12.402010,158,588
6,Hawaiian Airlines Inc.,HA,76272,0.484739,7807302.0,102.361312,10.929568,6454460.0,84.624240,6.825139,48249045,632.591842,2.016966,60,171
7,American Eagle Airlines Inc.,MQ,294632,9.632043,25814239.0,87.615191,15.712173,18673318.0,63.378445,8.615419,124427633,422.315407,6.110663,816,15025
8,Spirit Air Lines,NK,117379,15.683274,18028119.0,153.588964,14.356307,15245550.0,129.883114,9.390794,115649669,985.267118,14.202285,182,2004
9,Skywest Airlines Inc.,OO,588353,7.678231,58099335.0,98.749110,17.847826,43638869.0,74.171236,6.810587,292277393,496.772164,5.731005,1579,9960


In [ ]:
#insert this table into the sqlite database to perform further queries
df_airline_delay.to_sql(name='df_airline_delay', con=engine, if_exists='replace', index=False)
#check it worked
engine.table_names()

In [16]:
df_flights = pd.read_csv("flights.csv")
df_flights.head()

C:\Users\chris\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
df_flights.groupby(["AIRLINE"]).agg({"ARRIVAL_DELAY":['mean']})

,ARRIVAL_DELAY
,mean
AIRLINE,
AA,3.451372
AS,-0.976563
B6,6.677861
DL,0.186754
EV,6.585379
F9,12.504706
HA,2.023093
MQ,6.457873


In [18]:
df_flights.shape

(5819079, 31)